In [ ]:
import os
import numpy as np 
import pandas as pd 
import dropbox
from io import StringIO
from datetime import datetime

In [ ]:
#Modificar el año y las rutas para escribir los datos
#El path para promedios diarios se encuentra casi al final
anio="2016"
path_promedios="C:/path/a/promedioAnual"+anio+".csv"
path_promedios_month="C:/path/a/promedioMensual"+anio+".csv"
path_lista_puntos="C:/path/puntos/puntos_mar.npy"
path_lista_puntos_islas="C:/path/puntos/puntos_islas.npy"

#Ruta a datos en Dropbox
carpeta_dropbox="Datos_NSRDB_Colombia/"+anio
listaPuntos=np.load(path_lista_puntos)
listaPuntosIslas=np.load(path_lista_puntos_islas)
print("Tamaño lista puntos principal: ",len(listaPuntos))
print("Tamaño lista puntos islas: ",len(listaPuntosIslas))

In [ ]:
#Quitar coincidentes entre listaPuntosIslas y listaPuntos
#No se debe modificar
dic_listaPuntos={}
listaPuntosConcat= np.concatenate((listaPuntos,listaPuntosIslas))
listaPuntosConcatSinRep=[]
for i in range(len(listaPuntosConcat)):
  nombre_punto=str(listaPuntosConcat[i][0])+','+str(listaPuntosConcat[i][1])
  if (nombre_punto not in dic_listaPuntos):
    dic_listaPuntos[nombre_punto]=nombre_punto
    listaPuntosConcatSinRep.append(listaPuntosConcat[i])
print(len(listaPuntosConcatSinRep))
listaPuntos = listaPuntosConcatSinRep

In [ ]:
#Token para acceso a la carpeta de Dropbox - no es necesario modificarlo
dbx = dropbox.Dropbox('W3s3A8m5DCAAAAAAAAAAYJJbBNEO5yZthmR6Cju3UvMvNfW_S6m15HgdZKIyNgSB')

In [ ]:
#Arreglos que guardan las entradas de cada tabla
#Este script genera promedios diarios, mensuales y anuales

#Datos diarios
#Cada entrada es un promedio
lat_d = [] #Latitud
lon_d = [] #Longitud
mon_d = [] #Mes
day_d = [] #Dia
GHI_d = [] #Global horizontal
DHI_d = [] #Directa horizontal
DNI_d = [] #Directa normal
WS_d = [] #Velocidad del viento
T_d = [] #Temperatura
SZA_d = [] #Elevacion solar

#Datos mensuales
lat_m = []
lon_m = []
mon_m = []
GHI_m = []
DHI_m = []
DNI_m = []
WS_m = []
T_m = []
SZA_m = []

#Datos anuales
lat_y = []
lon_y = []
ele_y = []
GHI_y = []
DHI_y = []
DNI_y = []
WS_y = []
T_y = []
SZA_y = []

#Los promedios mensuales se guardan en un solo archivo para todos los puntos, igual con los promedios anuales
#como solo se tiene un archivo para mes y uno para año, se inicializan los headers de una vez
df_y = pd.DataFrame([], columns=['latitude','longitude','elevation','GHI','DHI','DNI','Wind Speed','Temperature','Solar Zenith Angle'])
df_y.to_csv(path_promedios, index=False)

df_m = pd.DataFrame([], columns=['latitude','longitude','month','GHI','DHI','DNI','Wind Speed','Temperature','Solar Zenith Angle'])
df_m.to_csv(path_promedios_month, index=False)

#Se inicializa el tiempo para saber el progreso del script
start = datetime.now()
print("Started at: {}".format(datetime.now().strftime("%d/%m/%Y %H:%M:%S")))

#Se recorren los puntos
curr = 0
len_lista_puntos = len(listaPuntos)
while curr < len_lista_puntos:
    #----------------------lectura de datos desde dropbox----------------------
    punto = listaPuntos[curr]
    curr += 1
    csv_path=str(punto[0])+"_"+str(punto[1])+"_"+anio+".csv"
    metadata,res = dbx.files_download(path="/Datos_NSRDB_Colombia/"+anio+"/"+csv_path)

    s=str(res.content,'utf-8')
    data = StringIO(s) 
    file_temp=pd.read_csv(data)
    
    ###Listas de datos obtenidos
    latitude = file_temp['Latitude'][0]
    longitude = file_temp['Longitude'][0]
    elevation = file_temp['Elevation'][0]
    ###Debido al formato de los datos originales, se usan filas como 'location ID' para el mes
    ###porque asi estan ordenados los datos
    month = [int(i) for i in list(file_temp['Location ID'][2:])]
    day = [int(i) for i in list(file_temp['City'][2:])]
    
    GHI_rows = [float(i) for i in list(file_temp['Latitude'][2:])]
    DHI_rows = [float(i) for i in list(file_temp['Longitude'][2:])]
    DNI_rows = [float(i) for i in list(file_temp['Time Zone'][2:])]
    WS_rows = [float(i) for i in list(file_temp['Elevation'][2:])]
    T_rows = [float(i) for i in list(file_temp['Local Time Zone'][2:])]
    SZA_rows = [float(i) for i in list(file_temp['Clearsky DHI Units'][2:])]
    
    #----------------------inicializacion de sumatorias para promedios----------------------
    #Cada fila del archivo original tiene la informacion de cada variable en un periodo de media hora
    #Para sacar el promedio, primero se van sumando los valores por cada variable
    totales = [0,0,0,0,0,0]
    totales_month = [0,0,0,0,0,0]
    totales_day = [0,0,0,0,0,0]
    #cada punto tiene len(GHI_rows) de datos temporales (cada media hora por un año)
    #----------------------sumatorias----------------------
    #Se recorren los datos para un punto y para un año
    length_year = len(GHI_rows)
    for i in range(length_year):
        totales_day = [totales_day[0]+GHI_rows[i],
                       totales_day[1]+DHI_rows[i],
                       totales_day[2]+DNI_rows[i],
                       totales_day[3]+WS_rows[i],
                       totales_day[4]+T_rows[i],
                       totales_day[5]+SZA_rows[i]]
        
        totales_month = [totales_month[0]+GHI_rows[i],
                         totales_month[1]+DHI_rows[i],
                         totales_month[2]+DNI_rows[i],
                         totales_month[3]+WS_rows[i],
                         totales_month[4]+T_rows[i],
                         totales_month[5]+SZA_rows[i]]
        
        totales = [totales[0]+GHI_rows[i],
                   totales[1]+DHI_rows[i],
                   totales[2]+DNI_rows[i],
                   totales[3]+WS_rows[i],
                   totales[4]+T_rows[i],
                   totales[5]+SZA_rows[i]]
        # -------------------------------DIA-------------------------------
        #Un dia termina cuando se cuentan 48 datos, esta confirmacion se hace con el modulo 48
        #(i+1)%48==0 indica que se esta leyendo el dato 48 (indice 47)
        if((i+1)%48==0 and i!=0): #Termina un dia (48 intervalos de media hora)
            lat_d.append(latitude)
            lon_d.append(longitude)
            mon_d.append(month[i])
            day_d.append(day[i])
            GHI_d.append(totales_day[0]/48)
            DHI_d.append(totales_day[1]/48)
            DNI_d.append(totales_day[2]/48)
            WS_d.append(totales_day[3]/48)
            T_d.append(totales_day[4]/48)
            SZA_d.append(totales_day[5]/48)
            #se reinicia la cuenta
            totales_day = [0,0,0,0,0,0]
        # -------------------------------MES-------------------------------
        #Un mes puede tener entre 28 y 31 dias, por lo que se verifica el cambio de mes
        #en vez de la cantidad de datos
        if(i == 17519): #si ultimo mes
            length_month = day[i]*48
            lat_m.append(latitude)
            lon_m.append(longitude)
            mon_m.append(month[i])
            GHI_m.append(totales_month[0]/length_month)
            DHI_m.append(totales_month[1]/length_month)
            DNI_m.append(totales_month[2]/length_month)
            WS_m.append(totales_month[3]/length_month)
            T_m.append(totales_month[4]/length_month)
            SZA_m.append(totales_month[5]/length_month)
            #reinicia la cuenta
            totales_month = [0,0,0,0,0,0]
        else: #puedo verificar el siguiente mes
            if(month[i] != month[i+1]):
                length_month = day[i]*48
                lat_m.append(latitude)
                lon_m.append(longitude)
                mon_m.append(month[i])
                GHI_m.append(totales_month[0]/length_month)
                DHI_m.append(totales_month[1]/length_month)
                DNI_m.append(totales_month[2]/length_month)
                WS_m.append(totales_month[3]/length_month)
                T_m.append(totales_month[4]/length_month)
                SZA_m.append(totales_month[5]/length_month)
                #reinicia la cuenta
                totales_month = [0,0,0,0,0,0]
    # -------------------------------AÑO-------------------------------
    #En este caso solo se suman todas las entradas y se dividen por la cantidad total
    #Esto se hace para cada punto
    lat_y.append(latitude)
    lon_y.append(longitude)
    ele_y.append(elevation)
    GHI_y.append(totales[0]/length_year)
    DHI_y.append(totales[1]/length_year)
    DNI_y.append(totales[2]/length_year)
    WS_y.append(totales[3]/length_year)
    T_y.append(totales[4]/length_year)
    SZA_y.append(totales[5]/length_year)
    
    #Los promedios diarios se guardan como archivos separados, uno por cada punto y año
    #es esperable que los arreglos sean de longitud 365-366
    data_day = {'latitude': lat_d,
                 'longitude': lon_d,
                 'month': mon_d,
                 'day': day_d,
                 'GHI': GHI_d,
                 'DHI': DHI_d,
                 'DNI': DNI_d,
                 'Wind Speed': WS_d,
                 'Temperature': T_d,
                 'Solar Zenith Angle': SZA_d}
    df_day = pd.DataFrame(data_day, columns=['latitude','longitude','month','day','GHI','DHI','DNI','Wind Speed','Temperature','Solar Zenith Angle'])
    
    #ESTE PATH SE DEBE MODIFICAR
    path_promedios_day="C:/path/csv/"+str(anio)+"/"+str(punto[0])+"_"+str(punto[1])+"_diario.csv"
    df_day.to_csv(path_promedios_day, index=False)
    
    #Se reinician los arreglos diarios
    lat_d = []
    lon_d = []
    mon_d = []
    day_d = []
    GHI_d = []
    DHI_d = []
    DNI_d = []
    WS_d = []
    T_d = []
    SZA_d = []
    
    #Para evitar problemas de uso de memoria RAM (al escribir todo al final) o de uso de disco (al escribir por cada punto)
    #se hacen escrituras periodicas cada 57 puntos = 1091 sesiones de escritura
    if curr%57 == 0:
        data_month = {'latitude': lat_m,
                        'longitude': lon_m,
                        'month': mon_m,
                        'GHI': GHI_m,
                        'DHI': DHI_m,
                        'DNI': DNI_m,
                        'Wind Speed': WS_m,
                        'Temperature': T_m,
                        'Solar Zenith Angle': SZA_m}
        df_month = pd.DataFrame(data_month, columns=['latitude','longitude','month','GHI','DHI','DNI','Wind Speed','Temperature','Solar Zenith Angle'])
        #El archivo ya esta creado y tiene headers, por lo que se escribe en modo append
        df_month.to_csv(path_promedios_month, mode='a', header=False, index=False)
        
        #Se limpian los arreglos
        lat_m = []
        lon_m = []
        mon_m = []
        GHI_m = []
        DHI_m = []
        DNI_m = []
        WS_m = []
        T_m = []
        SZA_m = []

        
        data = {'latitude': lat_y,
                 'longitude': lon_y,
                 'elevation': ele_y,
                 'GHI': GHI_y,
                 'DHI': DHI_y,
                 'DNI': DNI_y,
                 'Wind Speed': WS_y,
                 'Temperature': T_y,
                 'Solar Zenith Angle': SZA_y}
        df = pd.DataFrame(data, columns=['latitude','longitude','elevation','GHI','DHI','DNI','Wind Speed','Temperature','Solar Zenith Angle'])
        df.to_csv(path_promedios, mode='a', header=False, index=False)
        
        lat_y = []
        lon_y = []
        ele_y = []
        GHI_y = []
        DHI_y = []
        DNI_y = []
        WS_y = []
        T_y = []
        SZA_y = []
    
    print("{} - {}% done ({} of {}) - Coordinate: {}".format(datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
                                                             round((curr/len_lista_puntos)*100,3),
                                                             curr,len_lista_puntos,punto), end="\r")

#Solo para saber cuando termina
end = datetime.now()
total_time = end - start
print("")
print("Done at: {} in {}".format(datetime.now().strftime("%d/%m/%Y %H:%M:%S"),total_time))